# Pipeline Movies Lab

### Introduction

In this lesson, we'll work through our machine learning process with categorical data.

### Loading our Data

Let's begin by loading up our data.

In [2]:
import pandas as pd 

url = "https://raw.githubusercontent.com/jigsawlabs-student/pipelines-and-transformers/master/6-pipelining-lab/imdb_movies_nan.csv"
movies_df = pd.read_csv(url, index_col = 0)

This time we have sorted the movies for you.

In [3]:
movies_df[:2]

,title,genre,budget,runtime,year,month,revenue
1108,Pinocchio,Animation,2600000.0,88.0,1940,2,84300000
862,Lolita,Drama,2000000.0,153.0,1962,6,9250000


In [4]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 1108 to 357
Data columns (total 7 columns):
title      2000 non-null object
genre      1916 non-null object
budget     1880 non-null float64
runtime    2000 non-null float64
year       2000 non-null int64
month      2000 non-null int64
revenue    2000 non-null int64
dtypes: float64(2), int64(3), object(2)
memory usage: 125.0+ KB


Let's split our data up front.

Assign the revenue column to the variable `y`, and the remaining columns except for title to the variable `X`.

In [5]:
y = movies_df['revenue']

In [8]:
y[:3]

# 1108    84300000
# 862      9250000
# 1125    71000000
# Name: revenue, dtype: int64


1108    84300000
862      9250000
1125    71000000
Name: revenue, dtype: int64

In [6]:

X = None

In [9]:
X.columns

# Index(['genre', 'budget', 'runtime', 'year', 'month'], dtype='object')

Index(['genre', 'budget', 'runtime', 'year', 'month'], dtype='object')

Ok, now it's time to split our data.

We already have our data sorted, but let's make sure that when we perform the split our data is split with the most recent data in the test set, and the next set of recent data in the validation set.   

> We'll do this for you.

In [341]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = False, test_size = .2)

X_validate, X_test, y_validate, y_test = train_test_split(X_test, y_test, shuffle = False, test_size = .5)

In [342]:
X_train.shape

# (1600, 5)

(1600, 5)

In [343]:
X_validate.shape

(200, 5)

In [344]:
X_test.shape

(200, 5)

### Coercing the Data

Let's take another look at our data.

In [345]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 1108 to 357
Data columns (total 5 columns):
genre      1916 non-null object
budget     1880 non-null float64
runtime    2000 non-null float64
year       2000 non-null int64
month      2000 non-null int64
dtypes: float64(2), int64(2), object(1)
memory usage: 93.8+ KB


We cannot train the model because we have null values in `genre` and in `budget`.  Let's start with budget.  We'll replace the missing values in the budget column with the mean, and add a `budget_is_na` column that indicates if the value was missing.

In [347]:
mapper = None

In [348]:
transformed_X = mapper.fit_transform(X)
transformed_X[:2]

# 	budget	budget_is_na
# 1108	2600000.0	False
# 862	2000000.0	False

,budget,budget_is_na
1108,2600000.0,False
862,2000000.0,False


In [349]:
transformed_X['budget'].isna().sum(), transformed_X['budget_is_na'].sum()

# (0, 120)

(0, 120)

### Handling Categorical Values

Working with the categorical variables is different.  We don't need to add a new column for the na values, as this can be taken care of with our one hot encoding.  So let's take care of the na values in the genre column by replacing them with the string `na`. 

In [350]:
from sklearn.preprocessing import OneHotEncoder

In [351]:
mapper = None

In [352]:
transformed_X = mapper.fit_transform(X)
transformed_X[:2]

# 	budget	budget_is_na	genre
# 1108	2600000.0	False	Animation
# 862	2000000.0	False	Drama

,budget,budget_is_na,genre
1108,2600000.0,False,Animation
862,2000000.0,False,Drama


In [353]:
transformed_X['genre'].value_counts()

# Action             483
# Drama              365
# Comedy             359
# Adventure          236
# Animation           93
# na                  84
# Fantasy             80
# Crime               76
# Thriller            73
# Horror              59
# Science Fiction     52
# Romance             40
# Name: genre, dtype: int64

Action             483
Drama              365
Comedy             359
Adventure          236
Animation           93
na                  84
Fantasy             80
Crime               76
Thriller            73
Horror              59
Science Fiction     52
Romance             40
Name: genre, dtype: int64

Ok, now let's add one hot encoding with our genre columns.

In [354]:
mapper = None

In [355]:
transformed_X = mapper.fit_transform(X)
transformed_X[:3]

# 	budget	budget_is_na	genre_x0_Action	genre_x0_Adventure	genre_x0_Animation	genre_x0_Comedy	genre_x0_Crime	genre_x0_Drama	genre_x0_Fantasy	genre_x0_Horror	genre_x0_Romance	genre_x0_Science Fiction	genre_x0_Thriller	genre_x0_na
# 1108	2600000.0	False	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
# 862	2000000.0	False	0.0	0.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0
# 1125	31115000.0	False	0.0	0.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0

,budget,budget_is_na,genre_x0_Action,genre_x0_Adventure,genre_x0_Animation,genre_x0_Comedy,genre_x0_Crime,genre_x0_Drama,genre_x0_Fantasy,genre_x0_Horror,genre_x0_Romance,genre_x0_Science Fiction,genre_x0_Thriller,genre_x0_na
1108,2600000.0,False,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
862,2000000.0,False,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1125,31115000.0,False,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### Handling the Rest

Now that we've taken care of our genre and budget columns, let's see if the rest of our columns are ready to add in there. 

In [359]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 1108 to 357
Data columns (total 5 columns):
genre      1916 non-null object
budget     1880 non-null float64
runtime    2000 non-null float64
year       2000 non-null int64
month      2000 non-null int64
dtypes: float64(2), int64(2), object(1)
memory usage: 93.8+ KB


Runtime, year and month each have only non-null values.  They are also already numeric.  So let's have our mapper include them.

In [360]:
mapper = None

In [361]:
transformed_X = mapper.fit_transform(X)


> Check that `runtime`, `year` and `month` are included as columns.

In [362]:
transformed_X.columns

# Index(['budget', 'budget_is_na', 'genre_x0_Action', 'genre_x0_Adventure',
#        'genre_x0_Animation', 'genre_x0_Comedy', 'genre_x0_Crime',
#        'genre_x0_Drama', 'genre_x0_Fantasy', 'genre_x0_Horror',
#        'genre_x0_Romance', 'genre_x0_Science Fiction', 'genre_x0_Thriller',
#        'genre_x0_na', 'runtime', 'year', 'month'],
#       dtype='object')

Index(['budget', 'budget_is_na', 'genre_x0_Action', 'genre_x0_Adventure',
       'genre_x0_Animation', 'genre_x0_Comedy', 'genre_x0_Crime',
       'genre_x0_Drama', 'genre_x0_Fantasy', 'genre_x0_Horror',
       'genre_x0_Romance', 'genre_x0_Science Fiction', 'genre_x0_Thriller',
       'genre_x0_na', 'runtime', 'year', 'month'],
      dtype='object')

In [363]:
transformed_X[:3]

# budget	budget_is_na	genre_x0_Action	genre_x0_Adventure	genre_x0_Animation	genre_x0_Comedy	genre_x0_Crime	genre_x0_Drama	genre_x0_Fantasy	genre_x0_Horror	genre_x0_Romance	genre_x0_Science Fiction	genre_x0_Thriller	genre_x0_na	runtime	year	month
# 1108	2600000.0	False	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	88.0	1940.0	2.0
# 862	2000000.0	False	0.0	0.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	153.0	1962.0	6.0
# 1125	31115000.0	False	0.0	0.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	248.0	1963.0	6.0


,budget,budget_is_na,genre_x0_Action,genre_x0_Adventure,genre_x0_Animation,genre_x0_Comedy,genre_x0_Crime,genre_x0_Drama,genre_x0_Fantasy,genre_x0_Horror,genre_x0_Romance,genre_x0_Science Fiction,genre_x0_Thriller,genre_x0_na,runtime,year,month
1108,2600000.0,False,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,88.0,1940.0,2.0
862,2000000.0,False,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,153.0,1962.0,6.0
1125,31115000.0,False,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,248.0,1963.0,6.0


### Training the model

Ok, copy our mapper below.

In [364]:
mapper = None

And then, we can apply the mapping to both our training and validation sets, train and score the model.

In [365]:
X_train_coerced = mapper.fit_transform(X_train)

In [366]:
X_validate_coerced = mapper.transform(X_validate)
X_validate_coerced[:2]

,budget,budget_is_na,genre_x0_Action,genre_x0_Adventure,genre_x0_Animation,genre_x0_Comedy,genre_x0_Crime,genre_x0_Drama,genre_x0_Fantasy,genre_x0_Horror,genre_x0_Romance,genre_x0_Science Fiction,genre_x0_Thriller,genre_x0_na,runtime,year,month
597,70000000.0,False,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,88.0,2012.0,1.0
800,58000000.0,False,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,125.0,2012.0,1.0


And train our model on X_train_coerced.

In [368]:
model = None

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [369]:
model.score(X_validate_coerced, y_validate)
# 0.537861413573713

0.5301554092507477

### Feature Selection

Next we can move onto looking at the importance of our features.

In [370]:
from eli5.sklearn import PermutationImportance
import eli5

perm = PermutationImportance(model).fit(X_validate_coerced, y_validate)
exp_df = eli5.explain_weights_df(perm, feature_names = list(X_train_coerced.columns))
exp_df[:15]

Let's try validating again, but this time with just the top five most important features.

In [373]:
exp_df['feature'].values[:4]

array(['budget', 'budget_is_na', 'runtime', 'genre_x0_Animation'],
      dtype=object)

### Continuing Feature Engineering

In [374]:
top_features = X_train[['budget', 'runtime']]

Ok, now that we have identified our top features, we can focus our time on improving these features.  Let's start by exploring the budget and runtime, by plotting them in matplotlib.

In [10]:
# histogram here

<img src="./budget-runtime-hist.png" width="40%">

And then let's describe these features.

In [376]:
# described features

# 	budget	runtime
# count	1.497000e+03	1600.000000
# mean	6.022669e+07	113.091875
# std	4.048444e+07	21.658927
# min	2.500000e+01	0.000000
# 25%	3.200000e+07	98.000000
# 50%	5.000000e+07	109.500000
# 75%	7.500000e+07	124.000000
# max	3.800000e+08	254.000000

,budget,runtime
count,1.497000e+03,1600.000000
mean,6.022669e+07,113.091875
std,4.048444e+07,21.658927
min,2.500000e+01,0.000000
25%,3.200000e+07,98.000000
50%,5.000000e+07,109.500000
75%,7.500000e+07,124.000000
max,3.800000e+08,254.000000


Ok. it looks like both budget and runtime has some suspicious values.  View the lowest values of budget using `value_counts`.

In [377]:
bottom_budgets = None
bottom_budgets.iloc[:5]

# 25.0        1
# 28.0        1
# 30.0        1
# 110.0       1
# 500000.0    1
# Name: budget, dtype: int64

25.0        1
28.0        1
30.0        1
110.0       1
500000.0    1
Name: budget, dtype: int64

And let's do the same for `runtime`.

In [378]:
bottom_runtimes = None
bottom_runtimes.iloc[:5]

# 0.0     1
# 63.0    1
# 72.0    1
# 74.0    1
# 75.0    2
# Name: runtime, dtype: int64

0.0     1
63.0    1
72.0    1
74.0    1
75.0    2
Name: runtime, dtype: int64

Ok, so let's replace set a threshold for under 100,000 for budget, and under 40 minutes for runtime.  Use a custom transformer to do so.  Add it to your `DataFrameMapper` from above.

In [398]:
mapper = None

In [399]:
X_train_coerced = mapper.fit_transform(X_train)

In [400]:
(X_train_coerced['budget'] < 100000).sum()
# 0

0

In [394]:
# X_train_coerced['budget'].value_counts().sort_index()

In [401]:
(X_train_coerced['runtime'] < 40).sum()
# 0

0

* Coercing the target

Now let's drop rows with 0 values from `movies_df`.

In [316]:
pruned_movies_df = movies_df[~(movies_df['revenue'] == 0)]

### Wrapping Up

Select our X and y columns.

In [317]:
y = pruned_movies_df['revenue']
X = pruned_movies_df.drop(columns = ['title', 'revenue'])

In [319]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = False, test_size = .2)

X_validate, X_test, y_validate, y_test = train_test_split(X_test, y_test, shuffle = False, test_size = .5)

Copy our mapper code.

In [409]:
# copy latest mapper code along with FunctionTransformer and related functions

Fit and transform our train data, and transform our validate and test data.

In [405]:
X_train_coerced = mapper.fit_transform(X_train)
X_validate_coerced = mapper.fit_transform(X_validate)
X_test_coerced = mapper.fit_transform(X_test)

In [406]:
y_train.isna().sum()

0

In [407]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_coerced, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [408]:
model.score(X_validate_coerced, y_validate)

0.5292171950462577

In [414]:
from eli5.sklearn import PermutationImportance
import eli5

perm = PermutationImportance(model).fit(X_validate_coerced, y_validate)
exp_df = eli5.explain_weights_df(perm, feature_names = list(X_train_coerced.columns))
exp_df[:4]

,feature,weight,std
0,budget,0.757332,0.025831
1,budget_is_na,0.035654,0.010210
2,runtime,0.034215,0.003914
3,genre_x0_Animation,0.029332,0.003169


In [416]:
selected_cols = exp_df[:4].feature.values

In [417]:
model = LinearRegression()
model.fit(X_train_coerced[selected_cols], y_train)

model.score(X_test_coerced[selected_cols], y_test)

0.5299022656678686

### Resources

* [Sklearn Function Transformers](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_function_transformer.html)

* [Python Data Science ](https://python-data-science.readthedocs.io/en/latest/preprocess.html)